In [1]:
import pandas as pd
import numpy as np
import re
import whois

In [2]:
base_df = pd.read_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/dataset.csv')
print(base_df)

                                                     URL  URLType
0      http://mylust.com/videos/215137/my-raunchy-loo...        0
1      http://mic.com/articles/101766/one-perfect-twe...        0
2      http://stackexchange.com/newsletters/newslette...        0
3      http://bisaneema.org/%e0%a4%ad%e0%a4%be%e0%a4%...        1
4      http://skyrock.com/videos/eDJwODRhNzoxMw==-bug...        0
...                                                  ...      ...
46939  http://themeforest.net/item/artistas-modern-po...        0
46940  http://torcache.net/torrent/2EE5318B3923B273EF...        0
46941  http://grasslandhotel.com.vn/index.php/compone...        1
46942  http://conservativetribune.com/barack-obama-ju...        0
46943  http://sberbank.ru/moscow/ru/person/investment...        0

[46944 rows x 2 columns]


In [84]:
import datetime
from socket import *

def host_ft_extract(url):
    if not url:
        return
    
    feature_list = []
    url_dt = {}
    url_regex = '(?:http.*://)?(?P<host>[^:/ ]+).*'
    url_search = re.search(url_regex, url)
    hostname = url_search.group('host')

    try:
        domain = whois.whois(hostname)
    except whois.parser.PywhoisError:
        url_dt.update({
            "RegistryDate": pd.NaT,
            "ExpirationDate": pd.NaT,
            "HostCountry": None,
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        return df
    except timeout:
        url_dt.update({
            "RegistryDate": pd.NaT,
            "ExpirationDate": pd.NaT,
            "HostCountry": None,
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
    except gaierror:
        url_dt.update({
            "RegistryDate": pd.NaT,
            "ExpirationDate": pd.NaT,
            "HostCountry": None,
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
    except ConnectionResetError:
        url_dt.update({
            "RegistryDate": pd.NaT,
            "ExpirationDate": pd.NaT,
            "HostCountry": None,
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
    except ConnectionRefusedError:
        url_dt.update({
            "RegistryDate": pd.NaT,
            "ExpirationDate": pd.NaT,
            "HostCountry": None,
            "DomainAge": 0,
            "ExpYear": 0,
            "ExpMonth": 0,
            "ExpDay": 0
        })
        
        feature_list.append(url_dt)
        df = pd.DataFrame(feature_list)
        
        return df
        
        
    reg_date = domain.creation_date
    if isinstance(reg_date, list):
        url_dt.update({"RegistryDate": reg_date[0]})
    elif (reg_date is None) or (reg_date == ""):
        url_dt.update({"RegistryDate": pd.NaT})
    else:
        try:
            url_dt.update({"RegistryDate": pd.to_datetime(reg_date)})
        except ValueError:
            url_dt.update({"RegistryDate": pd.NaT})

    exp_date = domain.expiration_date
    if isinstance(exp_date, list):
        url_dt.update({"ExpirationDate": exp_date[0]})
    elif (exp_date is None) or (exp_date == ""):
        url_dt.update({"ExpirationDate": pd.NaT})
    else:
        try:
            url_dt.update({"ExpirationDate": pd.to_datetime(exp_date)})
        except ValueError:
            url_dt.update({"ExpirationDate": pd.NaT})

    country = domain.country
    url_dt.update({"HostCountry": country})

    if pd.isnull(url_dt["RegistryDate"]):
        domain_age = 0
        url_dt.update({"DomainAge": domain_age})
    else:
        domain_age = datetime.datetime.now() - url_dt["RegistryDate"]
        age_in_days = domain_age.days
        url_dt.update({"DomainAge": age_in_days})

    feature_list.append(url_dt)
    df = pd.DataFrame(feature_list)
    if pd.isnull(df["ExpirationDate"].any):
        df["ExpYear"] = float("NaN")
        df["ExpMonth"] = float("NaN")
        df["ExpDay"] = float("NaN")
    else:
        df["ExpYear"] = df["ExpirationDate"].dt.year
        df["ExpMonth"] = df["ExpirationDate"].dt.month
        df["ExpDay"] = df["ExpirationDate"].dt.day
    
    return df

In [49]:
dataframe_list = []
for idx, row in base_df.iterrows():
    print(row['URL'])
    feature_df = host_ft_extract(row['URL'])
    feature_df['URLType'] = row["URLType"]
    dataframe_list.append(feature_df)
    print(feature_df)
    
extracted_df = pd.DataFrame.concat(dataframe_list)
print(extracted_df)

http://mylust.com/videos/215137/my-raunchy-looking-wife-definitely-loves-missionary-position/
IS LIST
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2004-11-19 05:50:20 2021-11-19 05:50:20          PA       5594     2021   

   ExpMonth  ExpDay  URLType  
0        11      19        0  
http://mic.com/articles/101766/one-perfect-tweet-calls-out-the-hypocrisy-of-how-the-media-talks-about-black-violence
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://stackexchange.com/newsletters/newsletter?site=productivity.stackexchange.com
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://bisaneema.org/%e0%a4%ad%e0%a4%be%e0%a4%

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          CN          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        1  
http://icicibank.com/Personal-Banking/onlineservice/online-services/online-tax-payment/manage-tax.page
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          IN          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://kienthuc.net.vn/hoi-dap-ve-tuyen-sinh/phuong-an-thi-moi-co-lam-giam-ty-le-do-tot-nghiep-thpt-463742.html
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        0  
http://www.xj9599.com/app/member/indexx.php?path=pages&pages=yh&uid=f554ab9b50d41d8cd98fra3
IS LIST
         RegistryDate      ExpirationDate HostCountry  Dom

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://nypost.com/2015/05/13/rangers-strategy-for-dealing-with-trash-talking-alex-ovechkin/
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://9779.info/%E5%B0%91%E5%84%BF%E7%A7%8D%E5%AD%90%E7%B2%98%E8%B4%B4%E7%94%BB/
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          CN          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        1  
http://3cf.ru/2015/07/20/stoimost-nefti-wti-vpervye-s-aprelia-ypala-nije-50-za-barrel/
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          Na

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://hollywoodlife.com/2015/04/01/iggy-azalea-boob-job-10-year-old-boy-interview/
IS LIST
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 1999-09-29 06:03:23 2020-09-29 06:03:14          US       7472     2020   

   ExpMonth  ExpDay  URLType  
0         9      29        0  
http://atwiki.jp/wiki/%E3%83%B4%E3%82%A1%E3%83%B3%E3%82%AC%E3%83%BC%E3%83%89%E3%81%AE%E3%83%87%E3%83%83%E3%82%AD%E3%81%AE%E4%BD%9C%E3%82%8A%E6%96%B9
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
https://dribbble.com/shots/1952800-ViRail-Website-Redesign-with-Google-Material-Design
IS LIST
         Regi

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://sourceforge.net/directory/communications/add_facet_filter?facet=license&constraint=OSI-Approved+Open+Source+%3A%3A+GNU+Library+or+Lesser+General+Public+License+version+2.0+%28LGPLv2%29
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://schertzauto.com/wp-content/plugins/revslider/rs-plugin/css/settings.css?rev=4.6.0&ver=4.2.2
IS LIST
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2003-10-12 22:23:19 2021-10-12 22:23:19          US       5997     2021   

   ExpMonth  ExpDay  URLType  
0        10      12        1  
http://9779.info/%E4%BA%BA%E7%89%A9%E6%8A%BD%E8%B1

...

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          CN          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        1  
http://ecnavi.jp/redirect/?url=http://px.a8.net/svt/ejp?a8mat=2HHQ5I+4Y0MNM+2B8O+NTZCH&id1=UI%user_id%IT59216FR13
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          JP          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://techcrunch.com/2015/05/13/microsoft-picked-a-name-for-the-smartphone-friendly-build-of-windows-10/
IS LIST
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2005-06-10 22:56:20 2021-06-10 22:56:20          US       5390     2021   

   ExpMonth  ExpDay  URLType  
0         6      10        0  
http://stackoverflow.com/questions/30226153/how-to-validate-uploaded-file-in-relate-model-in-asp-net-mvc
ISNT LIST
NULL R

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate           HostCountry  DomainAge  ExpYear  \
0          NaT            NaT  REDACTED FOR PRIVACY          0      NaN   

   ExpMonth  ExpDay  URLType  
0       NaN     NaN        0  
http://metro.co.uk/2015/05/13/14-films-you-should-totally-be-excited-about-from-cannes-film-festival-2015-5194551/
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://pelis24.com/peliculasvose/17074-supernatural-10x02-reichenbach-2005-online.html
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://techcrunch.com/2015/05/07/whos-viewed-your-posts-linkedin-adds-analytics-to-its-publishing-platform/
IS LIST
         Re

ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT          US          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://icicibank.com/nri-banking/new-age-banking/online-account-opening-process.page
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://pikabu.ru/tag/%D0%90%D1%80%D0%BD%D0%BE%D0%BB%D1%8C%D0%B4%20%D0%A8%D0%B2%D0%B0%D1%80%D1%86%D0%B5%D0%BD%D0%B5%D0%B3%D0%B3%D0%B5%D1%80/hot
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://tinnhanh360.net/khong-thay--dau-hieu-nguoi-yeu-nghi-ngo-su-trong-trang-cua-em.html
  RegistryDate ExpirationDate H

ISNT LIST
         RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2000-01-21 16:28:17            NaT          US       7357      NaN   

   ExpMonth  ExpDay  URLType  
0       NaN     NaN        0  
http://1337x.to/torrent/1160420/Marvels-Agents-of-S-H-I-E-L-D-S02E22-720p-HDTV-x264-AAC-Ozlem/
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://web.de/magazine/politik/gerechte-verteilung-bruessel-fluechtlings-quoten-30638040
ISNT LIST
NULL REG DATE
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://industrialtrainingzirakpur.com/asp-dotnet-training-in-zirakpur.html
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT 

ConnectionRefusedError: [Errno 61] Connection refused

In [55]:
extracted_df = pd.concat(dataframe_list, ignore_index=True)
extracted_df.reset_index()
print(extracted_df)

           RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0   2004-11-19 05:50:20 2021-11-19 05:50:20          PA       5594   2021.0   
1                   NaT                 NaT          US          0      NaN   
2                   NaT                 NaT          US          0      NaN   
3   2016-06-27 04:48:43 2020-06-27 04:48:43          IN       1356   2020.0   
4                   NaT                 NaT          FR          0      NaN   
..                  ...                 ...         ...        ...      ...   
834                 NaT                 NaT        None          0      0.0   
835                 NaT                 NaT          US          0      NaN   
836                 NaT                 NaT        None          0      0.0   
837 2003-03-28 01:07:35 2022-03-28 01:07:35          US       6196   2022.0   
838                 NaT                 NaT        None          0      0.0   

     ExpMonth  ExpDay  URLType  
0        11.0    1

In [56]:
extracted_df.to_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/domain_dataset.csv')

In [60]:
extracted_df['URLType'].value_counts()

0    624
1    215
Name: URLType, dtype: int64

In [69]:
benign_df = base_df[base_df.URLType.eq(0)]
benign_df = benign_df[:1000]
benign_df.reset_index()
display(benign_df)

,URL,URLType
0,http://mylust.com/videos/215137/my-raunchy-loo...,0
1,http://mic.com/articles/101766/one-perfect-twe...,0
2,http://stackexchange.com/newsletters/newslette...,0
4,http://skyrock.com/videos/eDJwODRhNzoxMw==-bug...,0
5,http://qz.com/404384/after-ten-days-on-the-mcd...,0
...,...,...
1320,http://torcache.net/torrent/5AAE72D1A57498B0FD...,0
1321,http://bdnews24.com/bangladesh/2015/05/13/megh...,0
1322,http://askubuntu.com/questions/622442/open-ter...,0
1323,http://atwiki.jp/wiki/%E6%BB%8B%E8%B3%80%20%E3...,0


In [70]:
malicious_df = base_df[base_df.URLType.eq(1)]
malicious_df = malicious_df[:1000]
malicious_df.reset_index()

frames = [benign_df, malicious_df]
domain_df = pd.concat(frames)
domain_df = domain_df.sample(frac=1).reset_index(drop=True)
display(domain_df)

,URL,URLType
0,http://www.58tpw.com/Template/Default_2009_08/...,1
1,http://mashable.com/people/sandra-gonzalez/201...,0
2,http://jmdongnan.com/daxibeimuban/template/jia...,1
3,http://pikabu.ru/tag/%D0%90%D1%80%D0%BD%D0%BE%...,0
4,http://maybank2u.com.my/mbb_info/m2u/public/pr...,0
...,...,...
1995,http://thenextweb.com/facebook/2015/04/27/make...,0
1996,http://caixa.gov.br/voce/poupanca-e-investimen...,0
1997,http://www.114633.com/cl/?module=System&method...,1
1998,http://1337x.to/torrent/1146750/Sage-the-Gemin...,0


In [71]:
domain_df["URLType"].value_counts()

1    1000
0    1000
Name: URLType, dtype: int64

In [85]:
new_dataframe_list = []
for idx, row in domain_df.iterrows():
    print(row['URL'])
    feature_df = host_ft_extract(row['URL'])
    feature_df['URLType'] = row["URLType"]
    new_dataframe_list.append(feature_df)
    print(feature_df)
    
domain_ft_df = pd.DataFrame.concat(new_dataframe_list)
print(domain_ft_df)

http://www.58tpw.com/Template/Default_2009_08/images/map_134.jpg
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2014-09-30 18:33:47 2020-09-30 18:33:47          CN       1992     2020   

   ExpMonth  ExpDay  URLType  
0         9      30        1  
http://mashable.com/people/sandra-gonzalez/2015/05/09/orphan-black-season-3-episode-4-recap
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2005-07-19 14:50:34 2026-07-19 14:50:34          US       5352     2026   

   ExpMonth  ExpDay  URLType  
0         7      19        0  
http://jmdongnan.com/daxibeimuban/template/jianfei/images/css.css
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2016-08-08 18:53:41 2020-08-08 18:53:41          CN       1314     2020   

   ExpMonth  ExpDay  URLType  
0         8       8        1  
http://pikabu.ru/tag/%D0%90%D1%80%D0%BD%D0%BE%D0%BB%D1%8C%D0%B4%20%D0%A8%D0%B2%D0%B0%D1%80%D1%86%D0%B5%D0%BD%D0%B5%D0%B3%D0%B3

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2012-03-06 20:59:19 2021-03-06 20:59:19          CN       2930     2021   

   ExpMonth  ExpDay  URLType  
0         3       6        1  
http://9779.info/%E8%91%97%E5%90%8D%E5%A3%81%E7%94%BB%E4%BD%9C%E5%93%81/
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2010-06-22 06:38:59 2023-06-22 06:38:59          CN       3554     2023   

   ExpMonth  ExpDay  URLType  
0         6      22        1  
http://3cf.ru/2015/07/21/hokkei-nhl-podtverdila-chto-zaiavki-na-vstyplenie-v-ligy-podali-tolko-kvebek-i-las-vegas/
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        1  
http://schertzauto.com/wp-content/plugins/revslider/rs-plugin/css/settings.css?rev=4.6.0&ver=4.2.2
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2003-10

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 1997-03-07 05:00:00 2021-03-08 05:00:00          US       8409     2021   

   ExpMonth  ExpDay  URLType  
0         3       8        0  
http://extratorrent.cc/torrent_download/4191069/The.Lizzie.Borden.Chronicles.S01E06.WEBRip.x264-CraiovaFly+%5BReUpload%5D.torrent
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2010-12-03 12:08:35 2023-12-03 12:08:35          BS       3390     2023   

   ExpMonth  ExpDay  URLType  
0        12       3        0  
http://olx.in/hi/m/item/laptop-80-gb-ide-hard-disk-are-available-IDUV945.html?code=bd4a7dc7d3
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2006-02-22 11:21:13 2021-02-22 11:21:13          NL       5135     2021   

   ExpMonth  ExpDay  URLType  
0         2      22        0  
http://www.lxytzx.com/bjfc/UploadFiles_7694/201011/2010111709535569.jpg
         RegistryDate      ExpirationDate HostCountry

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2018-06-25 05:49:58 2020-06-25 05:49:58          CN        629     2020   

   ExpMonth  ExpDay  URLType  
0         6      25        1  
http://pornsharing.com/thick-ass-rose-red-tyrell-does-the-splits-with-no-panties_v78320
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        0  
http://9779.info/%E4%B8%80%E5%B9%B4%E7%BA%A7%E8%B4%B4%E7%94%BB%E5%9B%BE%E7%89%87%E6%AC%A3%E8%B5%8F/
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2010-06-22 06:38:59 2023-06-22 06:38:59          CN       3554     2023   

   ExpMonth  ExpDay  URLType  
0         6      22        1  
http://ap.org/Content/AP-In-The-News/2012/Simmons-ex-AP-bureau-chief-and-reporter-dies
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 1991-07-25 04:00:00

  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        0  
http://torrentdn.com/bbs/s.php?bo_table=torrent_util&wr_id=18263&k=%ED%8F%AC%ED%86%A0%EC%83%B5&page=1
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2015-03-24 07:05:16 2020-03-24 07:05:16          US       1818     2020   

   ExpMonth  ExpDay  URLType  
0         3      24        0  
http://bisaneema.org/%e0%a4%ad%e0%a4%be%e0%a4%97%e0%a4%b5%e0%a4%a4-%e0%a4%9c%e0%a5%8d%e0%a4%9e%e0%a4%be%e0%a4%a8-%e0%a4%af%e0%a4%9c%e0%a5%8d%e0%a4%9e-%e0%a4%b8%e0%a4%ae%e0%a4%bf%e0%a4%a4%e0%a4%bf/
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2016-06-27 04:48:43 2020-06-27 04:48:43          IN       1357     2020   

   ExpMonth  ExpDay  URLType  
0         6      27        1  
http://serverfault.com/questions/688477/how-does-squid-understand-the-dest

...

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 1995-11-01 05:00:00 2021-02-02 11:59:59          US       8901     2021   

   ExpMonth  ExpDay  URLType  
0         2       2        0  
http://industrialtrainingzirakpur.com/asp-dotnet-training-in-zirakpur.html
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        1  
http://golfsource.us/index.php?option=com_content&view=article&id=7&Itemid=1
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        1  
http://extratorrent.cc/torrent_download/4189628/Fast+%26+Furious+7+%282015%29+Esub+720p+HDRip+%5BDual+Audio%5D%5BHindi+-+English%5D%7EStroMSiD+%5BTEAM+warriors%5D.torrent
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2008-08-02 19:06:27 2020-08-02 19:06:27          PA       4242     2020   

   ExpMonth  ExpDay  URLType  
0         8       2        0  
http://www.xpj5988.com/gamerule_baccart.htm;jsessionid=A77C20A90C6790855CB2882AD06183ED
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2014-11-03 12:17:56 2021-11-03 12:17:56          US       1959     2021   

   ExpMonth  ExpDay  URLType  
0        11       3        1  
https://eksisozluk.com/sozlukculerin-ben-de-oradaydim-dedigi-olaylar--4493165?focusto=51290121
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2001-07-13 08:33:58 2025-11-09 05:23:07          TR       6820     2025   

   ExpMonth  ExpDay  URLType  
0        11       9        0  
http://3cf.ru/2015/07/21/kiev-i-opolchency-dogovorilis-ob-otvode-vooryjenii-do-100-mm/
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0   

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2010-06-22 06:38:59 2023-06-22 06:38:59          CN       3554     2023   

   ExpMonth  ExpDay  URLType  
0         6      22        1  
http://kenh14.vn/xa-hoi/muon-mac-le-roi-oi-hai-duong-ban-chay-o-ha-noi-20150107043314968.chn
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0        0         0   

   ExpDay  URLType  
0       0        0  
http://fanpage.gr/wp-content/plugins/contact-form-7/includes/js/jquery.form.min.js?ver=3.51.0-2014.06.20
  RegistryDate ExpirationDate HostCountry  DomainAge  ExpYear  ExpMonth  \
0          NaT            NaT        None          0      NaN       NaN   

   ExpDay  URLType  
0     NaN        0  
http://bjpgqsc.com/jypt/class/?0.html&page=2&showtj=&showhot=&key=
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2020-03-05 20:19:59 2021-03-05 20:19:59          CN  

         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2004-11-19 05:50:20 2021-11-19 05:50:20          PA       5595     2021   

   ExpMonth  ExpDay  URLType  
0        11      19        0  
http://www.5068.com/uploads/allimg/130821/81_130821132845_1_lit.jpg
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2003-10-09 18:43:44 2026-10-09 18:43:44        None       6001     2026   

   ExpMonth  ExpDay  URLType  
0        10       9        1  
http://torcache.net/torrent/56EE737750ACDD8817DE28BB1BBE9FD91136AE6F.torrent?title=[kickass.to]ex.machina.2015.dvdrip.xvid.evo
         RegistryDate      ExpirationDate HostCountry  DomainAge  ExpYear  \
0 2011-04-29 11:29:16 2020-04-29 11:29:16          US       3243     2020   

   ExpMonth  ExpDay  URLType  
0         4      29        0  
http://distractify.com/post/related/id/554ce7794a0c4b63184f8860/skip/30/limit/10/back/0
         RegistryDate      ExpirationDate HostCountry  DomainAge  

AttributeError: type object 'DataFrame' has no attribute 'concat'

In [86]:
dmn_ft_df = pd.concat(new_dataframe_list).reset_index()
display(dmn_ft_df)

,index,RegistryDate,ExpirationDate,HostCountry,DomainAge,ExpYear,ExpMonth,ExpDay,URLType
0,0,2014-09-30 18:33:47,2020-09-30 18:33:47,CN,1992,2020.0,9.0,30.0,1
1,0,2005-07-19 14:50:34,2026-07-19 14:50:34,US,5352,2026.0,7.0,19.0,0
2,0,2016-08-08 18:53:41,2020-08-08 18:53:41,CN,1314,2020.0,8.0,8.0,1
3,0,2009-04-03 20:00:00,2020-04-03 21:00:00,None,3998,2020.0,4.0,3.0,0
4,0,NaT,NaT,None,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
1995,0,2008-08-02 19:06:27,2020-08-02 19:06:27,PA,4242,2020.0,8.0,2.0,0
1996,0,NaT,NaT,None,0,NaN,NaN,NaN,0
1997,0,2015-06-08 06:40:44,2020-06-08 06:40:44,US,1742,2020.0,6.0,8.0,1
1998,0,NaT,NaT,None,0,NaN,NaN,NaN,0


In [87]:
dmn_ft_df.to_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/domain_feature_dataset.csv')

613

In [172]:
def replace_missing_values(df):
    default_reg = pd.to_datetime('2009-01-01 00:00:00')
    default_exp = pd.to_datetime('2020-08-01 00:00:00')
    domain_age = (datetime.datetime.now() - default_reg).days
    
    df['ExpYear'] = df.apply(
        lambda row: default_reg.year if (pd.isnull(row['ExpirationDate'])) else row['ExpYear'],
        axis=1
    )
    
    df['ExpMonth'] = df.apply(
        lambda row: default_exp.month if (pd.isnull(pd.to_datetime(row['ExpirationDate']))) else row['ExpMonth'],
        axis=1
    )
        
    df['ExpDay'] = df.apply(
        lambda row: default_exp.day if (pd.isnull(pd.to_datetime(row['ExpirationDate']))) else row['ExpDay'],
        axis=1
    )
    
    df['RegYear'] = df.apply(
        lambda row: default_reg.year if (pd.isnull(row['RegistryDate'])) else row['RegYear'],
        axis=1
    )
    
    df['RegMonth'] = df.apply(
        lambda row: default_exp.month if (pd.isnull(pd.to_datetime(row['RegistryDate']))) else row['RegMonth'],
        axis=1
    )
        
    df['RegDay'] = df.apply(
        lambda row: default_exp.day if (pd.isnull(pd.to_datetime(row['RegistryDate']))) else row['RegDay'],
        axis=1
    )
    
    df['DomainAge'] = df.apply(
        lambda row: domain_age if pd.isnull(pd.to_datetime(row['RegistryDate'])) else row['DomainAge'],
        axis=1
    )
    
    df['RegistryDate'] = df.apply(
        lambda row: default_reg if pd.isnull(pd.to_datetime(row['RegistryDate'])) else row['RegistryDate'],
        axis=1
    )
    
    df['ExpirationDate'] = df.apply(
        lambda row: default_exp if pd.isnull(pd.to_datetime(row['ExpirationDate'])) else row['ExpirationDate'],
        axis=1
    )

    df['HostCountry'] = df.apply(
        lambda row: 'US' if pd.isnull(row['HostCountry']) else row['HostCountry'],
        axis=1
    )

In [182]:
new_df = pd.read_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/domain_feature_dataset.csv', index_col=[0])
new_df.drop(['index'], axis=1, inplace=True)
display(new_df.columns)
new_df['RegistryDate'] = pd.to_datetime(new_df['RegistryDate'])
new_df['ExpirationDate'] = pd.to_datetime(new_df['ExpirationDate'])
print(new_df.dtypes)
    
replace_missing_values(new_df)
display(new_df)

Index(['RegistryDate', 'ExpirationDate', 'HostCountry', 'DomainAge', 'ExpYear',
       'ExpMonth', 'ExpDay', 'URLType'],
      dtype='object')

RegistryDate      datetime64[ns]
ExpirationDate    datetime64[ns]
HostCountry               object
DomainAge                  int64
ExpYear                  float64
ExpMonth                 float64
ExpDay                   float64
URLType                    int64
dtype: object


,RegistryDate,ExpirationDate,HostCountry,DomainAge,ExpYear,ExpMonth,ExpDay,URLType
0,2014-09-30 18:33:47,2020-09-30 18:33:47,CN,1992,2020.0,9.0,30.0,1
1,2005-07-19 14:50:34,2026-07-19 14:50:34,US,5352,2026.0,7.0,19.0,0
2,2016-08-08 18:53:41,2020-08-08 18:53:41,CN,1314,2020.0,8.0,8.0,1
3,2009-04-03 20:00:00,2020-04-03 21:00:00,US,3998,2020.0,4.0,3.0,0
4,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0
...,...,...,...,...,...,...,...,...
1995,2008-08-02 19:06:27,2020-08-02 19:06:27,PA,4242,2020.0,8.0,2.0,0
1996,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0
1997,2015-06-08 06:40:44,2020-06-08 06:40:44,US,1742,2020.0,6.0,8.0,1
1998,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0


In [183]:
new_df.to_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/domain_dataset_full.csv')

In [196]:
dmn_df = pd.read_csv('/Users/chrismclearnon/Developer/QUB-CSC3032-FYP/src/data/domain_dataset_full.csv')
display(dmn_df)

,Unnamed: 0,RegistryDate,ExpirationDate,HostCountry,DomainAge,ExpYear,ExpMonth,ExpDay,URLType
0,0,2014-09-30 18:33:47,2020-09-30 18:33:47,CN,1992,2020.0,9.0,30.0,1
1,1,2005-07-19 14:50:34,2026-07-19 14:50:34,US,5352,2026.0,7.0,19.0,0
2,2,2016-08-08 18:53:41,2020-08-08 18:53:41,CN,1314,2020.0,8.0,8.0,1
3,3,2009-04-03 20:00:00,2020-04-03 21:00:00,US,3998,2020.0,4.0,3.0,0
4,4,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0
...,...,...,...,...,...,...,...,...,...
1995,1995,2008-08-02 19:06:27,2020-08-02 19:06:27,PA,4242,2020.0,8.0,2.0,0
1996,1996,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0
1997,1997,2015-06-08 06:40:44,2020-06-08 06:40:44,US,1742,2020.0,6.0,8.0,1
1998,1998,2009-01-01 00:00:00,2020-08-01 00:00:00,US,4092,2009.0,8.0,1.0,0


In [197]:
dmn_df = dmn_df.drop(['RegistryDate', 'ExpirationDate'], axis=1)
dmn_df.reset_index()
display(dmn_df)

,Unnamed: 0,HostCountry,DomainAge,ExpYear,ExpMonth,ExpDay,URLType
0,0,CN,1992,2020.0,9.0,30.0,1
1,1,US,5352,2026.0,7.0,19.0,0
2,2,CN,1314,2020.0,8.0,8.0,1
3,3,US,3998,2020.0,4.0,3.0,0
4,4,US,4092,2009.0,8.0,1.0,0
...,...,...,...,...,...,...,...
1995,1995,PA,4242,2020.0,8.0,2.0,0
1996,1996,US,4092,2009.0,8.0,1.0,0
1997,1997,US,1742,2020.0,6.0,8.0,1
1998,1998,US,4092,2009.0,8.0,1.0,0
